In [0]:
%sql
select * from datamodelling.bronze.bronze_table

In [0]:
from pyspark.sql.functions import current_timestamp

load_df = spark.table("datamodelling.bronze.bronze_table").withColumn("load_timestamp", current_timestamp())


In [0]:
clean_data = load_df.dropna()
display(clean_data)

In [0]:
if spark.catalog.tableExists("datamodelling.default.source_data"):
    last_load_date=spark.sql("""select max(published_timestamp) from datamodelling.default.source_data""").collect()[0][0]
else:
    last_load_date='2010-01-01'

In [0]:
unique_data = clean_data.dropDuplicates(['course_id'])
unique_data.createOrReplaceTempView("silver_source")

In [0]:
%sql
create table if not exists datamodelling.silver.silver_table as select * from silver_source

In [0]:
%sql
merge into datamodelling.silver.silver_table as t 
using silver_source as s
on t.course_id = s.course_id
when matched then update set *
when not matched then insert *

In [0]:
# %sql
# insert into datamodelling.default.source_data values 
#     ('9999999', "Financial Modeling Bootcamp", "https://www.udemy.com/financial-modeling-bootcamp/", 'True',
#      '350', '5000', '45', '120', "Intermediate", '2.5', "2024-05-10T15:20:00Z", "Business Finance")


In [0]:
%sql
select *, published_timestamp
from datamodelling.default.source_data
where published_timestamp = (select max(published_timestamp) from datamodelling.default.source_data)